# 🌤️ ClimateCompass AI
### Multi-Agent Air Quality and Climate Risk Assistant (Built with LangGraph + RAG)

This project builds an **agentic AI system** that answers questions about **global air quality**, provides **health-related risk analysis**, and suggests **policy-oriented actions** — powered by data, reasoning, and retrieval.

**Built by:** MANAV DESAI  
**Powered by:** Python · Pandas · LangGraph · LangChain · OpenAI API · Chroma VectorStore


## 🧭 Overview

ClimateCompass AI combines structured environmental data and domain knowledge to act as an intelligent **AI coach for air quality**.

The system integrates:
- **A Data Agent** → performs numerical analysis on air-quality datasets.
- **A RAG Agent** → retrieves factual knowledge about AQI categories and health risks.
- **A Planner Agent** → turns insights into actionable recommendations.
- **A Router Agent** → classifies each user query and sends it to the right expert agent.

Together, they provide a conversational interface for understanding and mitigating pollution impacts.


## 🌐 Dataset

Dataset: [World Air Quality Index by City and Coordinates](https://www.kaggle.com/datasets/adityaramachandran27/world-air-quality-index-by-city-and-coordinates)

Extended dataset (`df_big`) includes:

| Column | Description |
|:-------|:-------------|
| **Country** | Name of the country |
| **City** | City name |
| **AQI Value** | Air Quality Index |
| **AQI Category** | AQI band (Good, Moderate, Unhealthy, etc.) |
| **CO AQI Value / Category** | Carbon Monoxide level |
| **Ozone AQI Value / Category** | Ozone level |
| **NO₂ AQI Value / Category** | Nitrogen Dioxide level |
| **PM2.5 AQI Value / Category** | Fine particulate matter |
| **lat / lng** | Geographical coordinates |

This dataset allows analysis by **city, country, and pollutant**, enabling AI agents to compute risk insights and identify pollution trends globally.


## 🧩 System Architecture

The project follows an **agent-based LangGraph design**:

1. **Router Agent**  
   Detects whether a query mentions a *city*, *country*, or *metric* and routes it to:
   - Data Agent (numerical analysis)
   - RAG Agent (context retrieval)
   - Planner Agent (recommendations)
   - QA Agent (fallback LLM)

2. **Data Agent**  
   - Works directly on `df_big`  
   - Answers analytical queries (e.g., “Top 5 cities by AQI”, “Which country has most cities above 100 AQI?”)

3. **RAG Agent**  
   - Retrieves structured guidance on AQI health impacts and climate connections using a vector store (Chroma).

4. **Planner Agent**  
   - Combines numerical insights + retrieved context to produce practical recommendations for individuals and city planners.

5. **QA Agent**  
   - Handles general non-data questions conversationally.

All agents are orchestrated through **LangGraph**, maintaining a shared `State` of conversation and context.


In [1]:

# CLIMATE - AI

# INSTALL DEPENDENCIES
!pip install -q kaggle langgraph langchain langchain-community langchain-openai chromadb tiktoken openai python-dotenv

import os
import pandas as pd
from google.colab import files

# LOADING DATASET FROM KAGGLE

print("📁 Upload your kaggle.json file from Kaggle (Account -> Create New API Token)")
uploaded = files.upload()  # choose kaggle.json

if "kaggle.json" not in uploaded:
    raise RuntimeError("kaggle.json not uploaded. Please upload it and rerun this cell.")

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Dataset: World Air Quality Index by City and Coordinates
# https://www.kaggle.com/datasets/adityaramachandran27/world-air-quality-index-by-city-and-coordinates
!mkdir -p ./aq_data
!kaggle datasets download -d adityaramachandran27/world-air-quality-index-by-city-and-coordinates -p ./aq_data
!unzip -o ./aq_data/world-air-quality-index-by-city-and-coordinates.zip -d ./aq_data

csv_files = [f for f in os.listdir('./aq_data') if f.endswith('.csv')]
if not csv_files:
    raise RuntimeError("No CSV found in ./aq_data after unzip.")
csv_path = os.path.join('./aq_data', csv_files[0])

df_big = pd.read_csv(csv_path)
print("✅ Dataset loaded:", csv_files[0])
print("Columns:", df_big.columns.tolist())
display(df_big.head())

# SECURELY SETTING API (I am using OPENAI)

import getpass

print("🔐 Enter your OpenAI API key (or press Enter to skip and use mock responses).")
key = getpass.getpass("OpenAI API key: ")
if key.strip():
    os.environ["OPENAI_API_KEY"] = key.strip()
    os.environ["CHAT_MODEL"] = "gpt-4o-mini"
    os.environ["EMBED_MODEL"] = "text-embedding-3-small"
else:
    print("⚠️ No API key provided. Using mock LLM responses.")

# LLM WRAPPER

from typing import List
from openai import OpenAI

def chat_llm(messages: List[dict]) -> str:
    """
    Uses OpenAI if OPENAI_API_KEY is set, otherwise returns simple mock responses.
    """
    if not os.getenv("OPENAI_API_KEY"):
        # Offline / mock behavior
        last_user = next((m for m in reversed(messages) if m.get("role") == "user"), {})
        text = (last_user.get("content") or "").lower()
        if "aqi" in text or "pm2.5" in text or "pm25" in text or "risk" in text:
            return "Mock risk explanation: high AQI and PM2.5 are unhealthy, especially for children, elderly, and those with lung/heart conditions."
        if "recommend" in text or "what should" in text or "policy" in text:
            return (
                "Mock recommendations:\n"
                "1) Limit outdoor activities on high-AQI days.\n"
                "2) Encourage public transport and cleaner fuels.\n"
                "3) Monitor AQI dashboards and issue alerts.\n"
            )
        if "summary" in text:
            return "Mock summary: Some cities have significantly higher AQI than others; actions are needed to reduce pollution."
        return "Mock reply: Ask about AQI, health risks, or recommendations."
    else:
        client = OpenAI()
        model = os.getenv("CHAT_MODEL", "gpt-4o-mini")
        resp = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0.2,
        )
        return resp.choices[0].message.content

# RAG BASIC KNOWLEDGE BASE (AQI & HEALTH)


from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import FakeEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
import tempfile

kb_texts = [
    """
    AQI categories (US EPA style):
    0-50: Good – little or no risk.
    51-100: Moderate – acceptable, but some pollutants may be a concern for very sensitive people.
    101-150: Unhealthy for Sensitive Groups – people with heart or lung disease, older adults, and children are at greater risk.
    151-200: Unhealthy – everyone may begin to experience health effects.
    201-300: Very Unhealthy – health alert; more serious effects for everyone.
    301-500: Hazardous – health warnings of emergency conditions.
    """,
    """
    PM2.5 (fine particulate matter) is strongly linked to health outcomes:
    Long-term exposure increases risk of heart and lung disease.
    Short-term spikes can trigger asthma attacks and other acute symptoms,
    especially for sensitive groups (children, older adults, people with pre-existing conditions).
    """,
    """
    Climate and air quality are connected:
    Weather patterns, temperature inversions, and lack of wind can trap pollutants.
    Heat waves can worsen ozone and PM levels.
    Policy responses often focus on reducing emissions from transport, industry, and power generation.
    """
]

kb_docs = [Document(page_content=t.strip()) for t in kb_texts]

def get_embeddings():
    if os.getenv("OPENAI_API_KEY"):
        return OpenAIEmbeddings(model=os.getenv("EMBED_MODEL", "text-embedding-3-small"))
    else:
        return FakeEmbeddings(size=1536)

vs_dir = tempfile.mkdtemp()
embeddings = get_embeddings()
kb_vs = Chroma.from_documents(kb_docs, embedding=embeddings, persist_directory=vs_dir)
retriever = kb_vs.as_retriever(search_kwargs={"k": 3})

def rag_lookup(query: str) -> str:
    docs = retriever.invoke(query)  # new LangChain retriever interface
    if not docs:
        return "No relevant guidance found."
    return "\n\n".join(d.page_content for d in docs)

# LANGGRAPH STATE & AGENTS

from langgraph.graph import StateGraph, END
from typing import TypedDict, Dict

class State(TypedDict, total=False):
    messages: List[dict]
    question: str
    df: "pd.DataFrame"
    data_insight: str
    rag_context: str
    recommendations: str
    route: str

import re

def router_node(state: State) -> State:
    """
    Route based on:
    - Country-level questions (with cities + AQI) -> data agent
    - If a city name from the dataset appears in the question -> data agent
    - Otherwise, use keyword-based routing.
    """
    q_full = state["messages"][-1]["content"]
    q = q_full.lower()
    state["question"] = q_full

    df_local = state.get("df")

    # If it's clearly a country-level question:
    # e.g. "Which country has most number of cities with AQI more than 100?"
    if "country" in q and ("city" in q or "cities" in q) and "aqi" in q:
        state["route"] = "data"
        print("[Router] Detected country-level question. Routing to DATA agent.")
        return state

    # If question mentions a known city name -> DATA agent
    if df_local is not None and "City" in df_local.columns:
        cities = df_local["City"].dropna().unique().tolist()
        # Match whole words to avoid partials; skip the city named "Most"
        for city in sorted(cities, key=lambda c: -len(str(c))):
            city_l = str(city).lower()
            if city_l == "most":  # avoid confusion with the English word "most"
                continue
            if re.search(r"\b" + re.escape(city_l) + r"\b", q):
                state["route"] = "data"
                print(f"[Router] Detected city '{city}' in question. Routing to DATA agent.")
                return state

    # Otherwise, fall back to keyword routing
    if any(x in q for x in [
        "which city", "top 5", "top five", "highest", "lowest",
        "compare", "average", "mean", "trend", "max", "min", "by aqi"
    ]):
        route = "data"
    elif any(x in q for x in ["health", "effect", "safe", "danger", "symptom", "risk"]):
        route = "rag"
    elif any(x in q for x in ["what should", "recommend", "policy", "next steps", "what can i do"]):
        route = "plan"
    else:
        route = "qa"

    state["route"] = route
    print(f"[Router] Sending question to: {route} agent")
    return state


def data_agent(state: State) -> State:
    """
    Uses df_big with columns:
    - Country, City
    - AQI Value, AQI Category
    - CO AQI Value, CO AQI Category
    - Ozone AQI Value, Ozone AQI Category
    - NO2 AQI Value, NO2 AQI Category
    - PM2.5 AQI Value, PM2.5 AQI Category
    - lat, lng

    Supports:
    - Country-level: "Which country has most number of cities with AQI more than 100?"
    - City-level summary: "What is the AQI for Indore?"
    - Rankings: "Show the top 5 cities by AQI", "Which city has the lowest NO2?"
    """
    df_local = state.get("df")
    question = state.get("question", "")
    q = question.lower()
    insight = ""

    if df_local is None:
        insight = "No dataset loaded."
    else:
        try:
            country_col = "Country"
            city_col = "City"
            aqi_value_col = "AQI Value"
            aqi_cat_col = "AQI Category"

            # COUNTRY-LEVEL THRESHOLD MODE
            # e.g. "Which country has most number of cities with AQI more than 100?"
            comp_match = re.search(
                r"(above|over|more than|greater than|exceeding|at least|>=|below|under|less than|<)\s*(\d+)",
                q
            )
            comparison = None
            threshold = None
            is_above = True
            if comp_match:
                comparison = comp_match.group(1)
                threshold = int(comp_match.group(2))
                above_words = {"above", "over", "more than", "greater than", "exceeding", "at least", ">=", ">"}
                is_above = comparison in above_words

            if "country" in q and ("city" in q or "cities" in q) and "aqi" in q and threshold is not None:
                if aqi_value_col not in df_local.columns or country_col not in df_local.columns:
                    insight = (
                        f"I couldn't find '{aqi_value_col}' and '{country_col}' columns in the dataset.\n"
                        f"Available columns: {list(df_local.columns)}"
                    )
                else:
                    if is_above:
                        df_thr = df_local[df_local[aqi_value_col] > threshold]
                        comp_phrase = f"above {threshold}"
                    else:
                        df_thr = df_local[df_local[aqi_value_col] < threshold]
                        comp_phrase = f"below {threshold}"

                    if df_thr.empty:
                        insight = f"No cities in the dataset have AQI {comp_phrase}."
                    else:
                        grp = (
                            df_thr
                            .groupby(country_col)[city_col]
                            .nunique()
                            .sort_values(ascending=False)
                        )
                        top_country = grp.index[0]
                        top_count = int(grp.iloc[0])

                        insight = (
                            f"The country with the most cities with AQI {comp_phrase} is "
                            f"**{top_country}** with **{top_count}** cities.\n\n"
                            f"Top 5 countries by number of cities with AQI {comp_phrase}:\n"
                        )
                        for ctry, count in grp.head(5).items():
                            insight += f"- {ctry}: {int(count)} cities\n"

                state["data_insight"] = insight
                msgs = state.get("messages", [])
                msgs.append({"role": "assistant", "content": insight})
                state["messages"] = msgs
                return state

            # METRIC SELECTION
            metric_map = {
                "aqi": "AQI Value",
                "overall aqi": "AQI Value",
                "pm2.5": "PM2.5 AQI Value",
                "pm25": "PM2.5 AQI Value",
                "no2": "NO2 AQI Value",
                "nitrogen dioxide": "NO2 AQI Value",
                "co": "CO AQI Value",
                "carbon monoxide": "CO AQI Value",
                "ozone": "Ozone AQI Value",
                "o3": "Ozone AQI Value",
            }

            if "no2" in q or "nitrogen dioxide" in q:
                metric_col = metric_map["no2"]
                metric_name = "NO2 AQI Value"
            elif "pm2.5" in q or "pm25" in q:
                metric_col = metric_map["pm2.5"]
                metric_name = "PM2.5 AQI Value"
            elif "co" in q or "carbon monoxide" in q:
                metric_col = metric_map["co"]
                metric_name = "CO AQI Value"
            elif "ozone" in q or "o3" in q:
                metric_col = metric_map["ozone"]
                metric_name = "Ozone AQI Value"
            else:
                metric_col = metric_map["aqi"]
                metric_name = "AQI Value"

            # IS THIS A RANKING QUERY?
            # e.g. "show the top 5 cities", "highest AQI", etc.
            is_ranking_query = any(
                kw in q for kw in ["top 5", "top five", "top", "highest", "lowest", "most polluted", "least polluted"]
            )

            # CITY DETECTION (ONLY IF NOT RANKING)
            city_in_question = None
            if (not is_ranking_query) and (city_col in df_local.columns):
                for city in df_local[city_col].dropna().unique():
                    city_l = str(city).lower()
                    # use word boundaries to avoid matching substrings like "ho" in "show"
                    if re.search(r"\b" + re.escape(city_l) + r"\b", q):
                        city_in_question = city
                        break

            # CITY-SPECIFIC SUMMARY
            if city_in_question is not None:
                subset = df_local[df_local[city_col] == city_in_question]
                if subset.empty:
                    insight = f"I couldn't find data for city '{city_in_question}' in the dataset."
                else:
                    lines = []
                    lines.append(f"For **{city_in_question}**, based on {len(subset)} records:")

                    def add_metric_summary(metric_col_name: str, label: str, category_col: str | None = None):
                        if metric_col_name in subset.columns:
                            mean_val = subset[metric_col_name].mean()
                            min_val = subset[metric_col_name].min()
                            max_val = subset[metric_col_name].max()
                            lines.append(
                                f"- {label}: avg {mean_val:.2f}, min {min_val:.2f}, max {max_val:.2f}"
                            )
                            if category_col and category_col in subset.columns:
                                top_cat = subset[category_col].value_counts().idxmax()
                                lines.append(f"  Most common {label} category: {top_cat}")

                    # Overall AQI
                    add_metric_summary("AQI Value", "AQI", "AQI Category")
                    # PM2.5
                    add_metric_summary("PM2.5 AQI Value", "PM2.5 AQI Value", "PM2.5 AQI Category")
                    # NO2
                    add_metric_summary("NO2 AQI Value", "NO2 AQI Value", "NO2 AQI Category")
                    # CO
                    add_metric_summary("CO AQI Value", "CO AQI Value", "CO AQI Category")
                    # Ozone
                    add_metric_summary("Ozone AQI Value", "Ozone AQI Value", "Ozone AQI Category")

                    insight = "\n".join(lines)

            # RANKING MODE (TOP / HIGHEST / LOWEST)
            else:
                # Sort direction
                if any(x in q for x in ["lowest", "least", "minimum", "min", "best"]):
                    ascending = True
                    direction = "lowest"
                else:
                    ascending = False
                    direction = "highest"

                if metric_col not in df_local.columns:
                    insight = (
                        f"I couldn't find column '{metric_col}' in the dataset.\n"
                        f"Available columns: {list(df_local.columns)}"
                    )
                else:
                    grp = (
                        df_local
                        .groupby(city_col)[metric_col]
                        .mean()
                        .sort_values(ascending=ascending)
                    )

                    insight = f"Top 5 cities by {direction} average {metric_name}:\n"
                    for c, val in grp.head(5).items():
                        insight += f"- {c}: {val:.2f}\n"

        except Exception as e:
            insight = f"Error while analyzing data: {e}"

    state["data_insight"] = insight
    msgs = state.get("messages", [])
    msgs.append({"role": "assistant", "content": insight})
    state["messages"] = msgs
    return state



# RAG AGENT
def rag_agent(state: State) -> State:
    question = state.get("question", "")
    context = rag_lookup(question)
    state["rag_context"] = context
    return state


# PLANNER AGENT
def plan_agent(state: State) -> State:
    question = state.get("question", "")
    data_insight = state.get("data_insight", "")
    rag_context = state.get("rag_context", "")

    prompt = f"""
You are ClimateCompass, an AI coach for air quality and climate risk.

User question:
{question}

Data insight (if any):
{data_insight}

Knowledge context:
{rag_context}

Provide:
1) A short plain-language answer.
2) 3–5 concrete recommendations (actions / policies / daily habits).
3) Note which groups are at higher risk, if relevant.
"""

    answer = chat_llm([
        {"role": "system", "content": "You are a helpful climate and air-quality advisor."},
        {"role": "user", "content": prompt},
    ])

    state["recommendations"] = answer
    msgs = state.get("messages", [])
    msgs.append({"role": "assistant", "content": answer})
    state["messages"] = msgs
    return state


# QA AGENT (FALLBACK)
def qa_agent(state: State) -> State:
    answer = chat_llm(state["messages"])
    state["messages"].append({"role": "assistant", "content": answer})
    return state


# BUILD GRAPH & CHAT LOOP

def build_app(df_loaded: pd.DataFrame):
    g = StateGraph(State)

    g.add_node("router", router_node)
    g.add_node("data", data_agent)
    g.add_node("rag", rag_agent)
    g.add_node("plan", plan_agent)
    g.add_node("qa", qa_agent)

    g.set_entry_point("router")

    g.add_conditional_edges(
        "router",
        lambda s: s["route"],
        {
            "data": "data",
            "rag": "rag",
            "plan": "plan",
            "qa": "qa",
        },
    )

    # data answers directly
    g.add_edge("data", END)
    # RAG -> planner
    g.add_edge("rag", "plan")
    # planner & qa end the turn
    g.add_edge("plan", END)
    g.add_edge("qa", END)

    app = g.compile()
    init_state: State = {"messages": [], "df": df_loaded}
    return app, init_state

app, state = build_app(df_big)
print("✅ ClimateCompass AI graph built.")


def chat_loop(app, state):
    print("🌍 ClimateCompass AI ready (ask about cities, AQI, risks, or actions).")
    print("Examples:")
    print("- What is the AQI for Indore?")
    print("- Show the top 5 cities by AQI.")
    print("- Which city has the lowest NO2 AQI Value?")
    print("- What are the health risks of high PM2.5?")
    print("- What recommendations would you give to city planners?\n")
    while True:
        q = input("You > ").strip()
        if q.lower() in {"exit", "quit"}:
            print("Bye!")
            break
        state["messages"].append({"role": "user", "content": q})
        state = app.invoke(state)
        print("Bot >", state["messages"][-1]["content"], "\n")

chat_loop(app, state)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.8/93.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.2/471.2 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.1 MB/s eta 0:00:

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/adityaramachandran27/world-air-quality-index-by-city-and-coordinates
License(s): CC-BY-NC-SA-4.0
  0% 0.00/372k [00:00<?, ?B/s]
100% 372k/372k [00:00<00:00, 791MB/s]
Archive:  ./aq_data/world-air-quality-index-by-city-and-coordinates.zip
  inflating: ./aq_data/AQI and Lat Long of Countries.csv  
✅ Dataset loaded: AQI and Lat Long of Countries.csv
Columns: ['Country', 'City', 'AQI Value', 'AQI Category', 'CO AQI Value', 'CO AQI Category', 'Ozone AQI Value', 'Ozone AQI Category', 'NO2 AQI Value', 'NO2 AQI Category', 'PM2.5 AQI Value', 'PM2.5 AQI Category', 'lat', 'lng']


,Country,City,AQI Value,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category,PM2.5 AQI Value,PM2.5 AQI Category,lat,lng
0,Russian Federation,Praskoveya,51,Moderate,1,Good,36,Good,0,Good,51,Moderate,44.7444,44.2031
1,Brazil,Presidente Dutra,41,Good,1,Good,5,Good,1,Good,41,Good,-5.2900,-44.4900
2,Brazil,Presidente Dutra,41,Good,1,Good,5,Good,1,Good,41,Good,-11.2958,-41.9869
3,Italy,Priolo Gargallo,66,Moderate,1,Good,39,Good,2,Good,66,Moderate,37.1667,15.1833
4,Poland,Przasnysz,34,Good,1,Good,34,Good,0,Good,20,Good,53.0167,20.8833


🔐 Enter your OpenAI API key (or press Enter to skip and use mock responses).
OpenAI API key: ··········
✅ ClimateCompass AI graph built.
🌍 ClimateCompass AI ready (ask about cities, AQI, risks, or actions).
Examples:
- What is the AQI for Indore?
- Show the top 5 cities by AQI.
- Which city has the lowest NO2 AQI Value?
- What are the health risks of high PM2.5?
- What recommendations would you give to city planners?

You > Show the top 5 cities by AQI.
[Router] Sending question to: data agent
Bot > Top 5 cities by highest average AQI Value:
- Dhanaura: 500.00
- Tynda: 500.00
- Malaut: 500.00
- Boksburg: 500.00
- Nohar: 500.00
 

You > top 5 countries with most cities with 100 plus aqi
[Router] Sending question to: data agent
Bot > Top 5 cities by highest average CO AQI Value:
- Durango: 133.00
- Miri: 67.00
- Boksburg: 49.00
- Alberton: 41.00
- Soweto: 36.00
 

You > Which country has most number of cities with AQI more than 100?
[Router] Detected country-level question. Routing to DA

## 🌎 Summary

ClimateCompass AI demonstrates how agentic architectures can **blend data analytics with language reasoning** to address climate and health problems.

Through modular AI agents, the system:
- Analyzes live data
- Retrieves factual climate knowledge
- Generates recommendations
- Engages in conversational Q&A

🧠 It’s a step toward intelligent, explainable systems that turn **environmental data into actionable guidance.**
